### Load data 

In [38]:
from datasets import load_dataset
data = load_dataset("uitnlp/vietnamese_students_feedback")

In [39]:
data#['train']

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

In [40]:
df_train = data['train'].to_pandas()
df_train.head(10)


,sentence,sentiment,topic
0,slide giáo trình đầy đủ .,2,1
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",2,0
2,đi học đầy đủ full điểm chuyên cần .,0,1
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",2,0
5,"giảng viên đảm bảo thời gian lên lớp , tích cự...",2,0
6,"em sẽ nợ môn này , nhưng em sẽ học lại ở các h...",1,3
7,"thời lượng học quá dài , không đảm bảo tiếp th...",0,1
8,"nội dung môn học có phần thiếu trọng tâm , hầu...",0,1
9,cần nói rõ hơn bằng cách trình bày lên bảng th...,0,1


In [41]:
data['train']['sentence'][47]

'những thông tin cần thiết như doubledot thời gian dealine , cách nộp bài thường không dặn lớp thực hành thứ hai .'

In [42]:
train_data = data['train']

In [43]:
import pandas as pd

In [44]:
train_data = train_data.rename_column("sentiment", "labels")
print(data)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})


In [45]:
train_data

Dataset({
    features: ['sentence', 'labels', 'topic'],
    num_rows: 11426
})

## Pre-processing data

### 1. Remove punctuation

In [46]:
import string

def remove_punctuation(input_string):
    translator = str.maketrans('', '', string.punctuation)
    cleaned_string = input_string.translate(translator)
    # Remove extra spaces
    cleaned_string = ' '.join(cleaned_string.split())
    return cleaned_string

print(remove_punctuation('giảng dạy hay , dễ hiểu , tận tình .'))
train_data = train_data.map(lambda example: {'sentence': remove_punctuation(example['sentence'])})


giảng dạy hay dễ hiểu tận tình


Map: 100%|██████████| 11426/11426 [00:00<00:00, 13338.29 examples/s]


### 2. Lower case

In [47]:
def lower(text):
    return text.lower()

#print(lower('.'))
train_data = train_data.map(lambda example: {'sentence': lower(example['sentence'])})

Map: 100%|██████████| 11426/11426 [00:00<00:00, 19519.25 examples/s]


### 3. Transform emoticons to word form

In [48]:
%pip install dictionary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [49]:
from Data_Processing.dictionary import emotion2wordform_dict, wordform2vnese_dict, mispelling_dict, translate_dict, number_dict

def emoticon2word(sentence):
    words = sentence.split()
    converted_words = [emotion2wordform_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "giáo viên dạy kiến tạo xã hội chủ yếu để học sinh tự học :)"
print("Converted sentence:", emoticon2word(sentence))

train_data = train_data.map(lambda example: {'sentence': emoticon2word(example['sentence'])})


Converted sentence: giáo viên dạy kiến tạo xã hội chủ yếu để học sinh tự học colonsmile


Map: 100%|██████████| 11426/11426 [00:00<00:00, 19882.84 examples/s]


### 4. Translate word form of emoticons into Viet Nam

In [50]:
def word_form2Vnese(sentence):
    words = sentence.split()
    converted_words = [wordform2vnese_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "giáo viên dạy kiến tạo xã hội chủ yếu để học sinh tự học colonsmile"
print("Converted sentence:", word_form2Vnese(sentence))

train_data = train_data.map(lambda example: {'sentence': word_form2Vnese(example['sentence'])})

Converted sentence: giáo viên dạy kiến tạo xã hội chủ yếu để học sinh tự học cười nhẹ


Map: 100%|██████████| 11426/11426 [00:00<00:00, 21380.53 examples/s]


### 5. Spelling correction with VietNamese

In [51]:
def translate2word(sentence, translate_dict):
    sentence = " " + sentence.strip() + " "
    for correct_word, incorrect_words in translate_dict.items():
        for incorrect_word in incorrect_words:
            sentence = sentence.replace(incorrect_word, correct_word)
    return sentence

sentence = 'bài lab này được upload lên proteus'
corrected_sentence = translate2word(sentence, translate_dict)
print("Converted sentence:", corrected_sentence)
train_data = train_data.map(lambda example: {'sentence': translate2word(example['sentence'], translate_dict)})

Converted sentence:  bài thực hành này được tải lên lên phần mềm 


Map: 100%|██████████| 11426/11426 [00:01<00:00, 10607.87 examples/s]


In [52]:
def mispell2word(sentence, mispelling_dict):
    # Thêm khoảng trắng vào đầu và cuối câu để đảm bảo thay thế từ đúng
    sentence = " " + sentence.strip() + " "
    
    # Thay thế các từ sai chính tả bằng từ đúng từ từ điển
    for correct_word, incorrect_words in mispelling_dict.items():
        for incorrect_word in incorrect_words:
            sentence = sentence.replace(incorrect_word, correct_word)
    
    # Loại bỏ khoảng trắng thừa ở đầu và cuối câu
    sentence = sentence.strip()
    
    return sentence

# Câu cần sửa lỗi chính tả
sentence = '.'
corrected_sentence = mispell2word(sentence, mispelling_dict)
print("Converted sentence:", corrected_sentence)
train_data = train_data.map(lambda example: {'sentence': mispell2word(example['sentence'], mispelling_dict)})


Converted sentence: .


Map: 100%|██████████| 11426/11426 [00:02<00:00, 3991.93 examples/s]


In [53]:
# Giả sử bạn đang sử dụng thư viện datasets của Hugging Face
from datasets import load_dataset

train_data = load_dataset("uitnlp/vietnamese_students_feedback")['train']

# Áp dụng hàm mispell2word cho mỗi câu trong tập dữ liệu
train_data = train_data.map(lambda example: {'sentence': mispell2word(example['sentence'], mispelling_dict)})

# Kiểm tra một vài ví dụ từ dữ liệu đã được sửa lỗi
for i in range(5):
    print(f"Original sentence: {train_data['sentence'][i]}")
    print(f"Corrected sentence: {mispell2word(train_data['sentence'][i], mispelling_dict)}")

Original sentence: slide giáo trình đầy đủ .
Corrected sentence: slide giáo trình đầy đủ .
Original sentence: nhiệt tình giảng dạy , gần gũi với sinh viên .
Corrected sentence: nhiệt tình giảng dạy , gần gũi với sinh viên .
Original sentence: đi học đầy đủ full điểm chuyên cần .
Corrected sentence: đi học đầy đủ full điểm chuyên cần .
Original sentence: chưa áp dụng công nghệ thông tin và các thiết bị hỗ trợ cho việc giảng dạy .
Corrected sentence: chưa áp dụng công nghệ thông tin và các thiết bị hỗ trợ cho việc giảng dạy .
Original sentence: thầy giảng bài hay , có nhiều bài tập ví dụ ngay trên lớp .
Corrected sentence: thầy giảng bài hay , có nhiều bài tập ví dụ ngay trên lớp .


In [54]:
def number2text(sentence):
    words = sentence.split()
    converted_words = [number_dict.get(word, word) for word in words]
    converted_sentence = ' '.join(converted_words)
    return converted_sentence

# Test the function
sentence = "bài tập của tôi được 10 điểm"
print("Converted sentence:", number2text(sentence))
train_data = train_data.map(lambda example: {'sentence': number2text(example['sentence'])})

Converted sentence: bài tập của tôi được mười điểm


In [55]:

def remove_stopwords(text, stopword_file=r'C:\Users\LAPTOP\Sentiment_Analysis_of_Student_Feedback\Data_Processing\stopword.txt'):
    # Read stopwords from the file with utf-8 encoding
    with open(stopword_file, 'r', encoding='utf-8') as file:
        stopwords = set(word.strip() for word in file)

    # Tokenize the input sentence into words
    words = text.split()

    # Remove the stopwords from the list of words
    filtered_words = [word for word in words if word.lower() not in stopwords]

    # Reconstruct the sentence without the stopwords
    text = ' '.join(filtered_words)

    return text
text = "bài tập của tôi được 10 điểm"
print("Removed stopwords:", remove_stopwords(text))
train_data = train_data.map(lambda example: {'sentence': remove_stopwords(example['sentence'])})

Removed stopwords: bài tập tôi 10 điểm


In [56]:
train_data = pd.DataFrame(train_data)
train_data.to_csv('processed_data.csv', index=False)

print("Data has been processed and saved to 'processed_data.csv'")

Data has been processed and saved to 'processed_data.csv'
